In [ ]:
import numpy as np
import os
from IPython.core.debugger import set_trace

In [ ]:
all_features = ["contrege_comp", "contrege_incomp", "incontrege", "bert_PCA_dims_15_contrege_incomp"]
predictions_dir = "predictions"
all_subjects = ['F','G','H','I','J','K','L','M','N']
num_sets = 5

In [ ]:
metrics = ["y_pred_0", "y_pred_1", "y_pred_2", "y_pred_3", "r2s", "y_test_0", "y_test_1", "y_test_2", "y_test_3"]
op = "avg"
add_suffix = "_pos_dep_tags_all_effort_based_metrics"

for metric in metrics:
    for feature in all_features:
        output_dir = ""
        output_dir = "{}/aggregated_{}".format(predictions_dir, feature) + add_suffix
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)
        for sub in all_subjects:
            agg = None
            if os.path.exists("{}/{}_{}.npy".format(output_dir, sub, metric)):
                continue
            if "test" in metric:
                print("Test data - {}. Copying it as is from the first set.".format(metric))
                data = np.load("{}/{}_set_{}{}/{}_{}.npy".format(predictions_dir, feature, 0, add_suffix, sub, metric))
                np.save("{}/{}_{}.npy".format(output_dir, sub, metric), data)
                continue
            for i in range(num_sets):
                data = np.load("{}/{}_set_{}{}/{}_{}.npy".format(predictions_dir, feature, i, add_suffix, sub, metric))
                if agg is None:
                    if op == "avg":
                        agg = data/num_sets
                    else:
                        agg = data
                else:
                    if op == "avg":
                        agg += data/num_sets
                    elif op == "min":
                        agg = np.minimum(agg, data)
                    elif op == "max":
                        agg = np.maximum(agg, data)
            np.save("{}/{}_{}.npy".format(output_dir, sub, metric), agg)